In [ ]:
# Standard library imports
import os

# Scientific computing libraries
import pandas as pd
import numpy as np

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Time series analysis libraries
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.eval_measures import rmse

# Deep learning libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models

# Utility libraries
import optuna # Hyperparameter optimization
from optuna.visualization import plot_parallel_coordinate, plot_slice, plot_param_importances
from tqdm import tqdm  # Progress bar visualization

# Custom libraries
from effKAN import KAN

In [ ]:
#read car data
data = pd.read_csv('./data/car_data.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
#check for missing values
data.isnull().sum()

In [ ]:
data = data.fillna('0')

# Graph

## Univariable Analysis

In [ ]:
sampled_data = data.sample(frac=0.01, random_state=42)

object_columns = sampled_data.select_dtypes(include=['object']).columns
object_columns = object_columns.drop(['Car_id','Customer Name','Date'])

if not os.path.exists('./graph/univariate'):
    os.makedirs('./graph/univariate')

# For Date Count
sales_counts = sampled_data['Date'].value_counts().sort_index().reset_index()
sales_counts.columns = ['Date', 'Sales']
plt.figure(figsize=(15, 5))
sns.barplot(x='Date', y='Sales', data=sales_counts, hue='Date')
plt.title('Sales Count by Date')
plt.xticks([])
plt.savefig('./graph/univariate/sales_count_by_date.png')
plt.show()

for i in range(len(object_columns)):
    plt.figure(figsize=(15, 5))
    sns.countplot(x=object_columns[i], data=sampled_data, hue=object_columns[i])
    plt.title(f'Count Plot for {object_columns[i]}')
    plt.xlabel(object_columns[i])
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.savefig(f'./graph/univariate/{object_columns[i]}_countplot.png')

plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
sns.kdeplot(x=sampled_data['Annual Income'], data=sampled_data)
plt.title(f'KDE Plot for Annual Income')
plt.xlabel('Annual Income')
plt.ylabel('KDE')
plt.xticks(rotation=90)
plt.savefig(f'./graph/univariate/Annual_Income_KDEplot.png')
plt.show()

## Bivariable Analysis

In [ ]:
if not os.path.exists('./graph/bivariate'):
    os.makedirs('./graph/bivariate')

for i in range(len(object_columns)):
    plt.figure(figsize=(15, 5))
    sns.violinplot(x=object_columns[i], y='Price ($)', data=sampled_data, hue=object_columns[i])
    plt.title(f'Violin Plot for {object_columns[i]} vs Price')
    plt.xlabel(object_columns[i])
    plt.ylabel('Price')
    plt.xticks(rotation=90)
    plt.savefig(f'./graph/bivariate/{object_columns[i]}_vs_price_violinplot.png')

plt.show()


In [ ]:
plt.figure(figsize=(15, 5))
sns.boxplot(x=sampled_data['Annual Income'], data=sampled_data)
plt.title(f'KDE Plot for Annual Income')
plt.xlabel('Annual Income')
plt.ylabel('KDE')
plt.xticks(rotation=90)
plt.savefig(f'./graph/bivariate/Annual_Income_KDE_plot.png')

plt.show()

## Multi-variate Analysis

In [ ]:
if not os.path.exists('./graph/multivariate'):
    os.makedirs('./graph/multivariate')

sns.pairplot(sampled_data)
plt.savefig('./graph/multivariate/pairplot.png')
plt.show()

# Convert, Encode, Normalization

## Convert `Date` to independent variable

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

data = data.drop(['Date'], axis=1)

## Encoding

In [ ]:
data = data.drop(['Car_id','Customer Name'], axis=1)

In [ ]:
object_columns = data.select_dtypes(include=['object']).columns

le = LabelEncoder()
for i in range(len(object_columns)):
    data[object_columns[i]] = le.fit_transform(data[object_columns[i]])

## Normalizing

In [ ]:
num_columns = data.select_dtypes(include=['int64', 'float64', 'int32']).columns
num_columns = num_columns.drop(['Price ($)'])

scaler = MinMaxScaler()
for i in range(len(num_columns)):
    data[num_columns[i]] = scaler.fit_transform(data[num_columns[i]].values.reshape(-1, 1))

# Features Analysis

In [ ]:
data.head()

In [ ]:
summary_stats = data.describe()

print("Summary of Statistics:")
summary_stats

In [ ]:
# Skewness and kurtosis
skewness = data.skew()
kurtosis = data.kurtosis()
# Display skewness and kurtosis values
print("\nSkewness:")
print(skewness)
print("\nKurtosis:")
print(kurtosis)

In [ ]:
# Correlation matrix
correlation_matrix = data.corr()

# Correlation heatmap
plt.figure(figsize=(20, 10))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", linewidths=0.5, fmt = ".3f")
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# Calculate Multicollinearity
y = data.drop(["Price ($)"], axis =1)
X = sm.add_constant(y)

# Calculate VIF for each variable
vif = pd.DataFrame()
vif["variable"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif)

In [ ]:
high_vif_variables = vif[vif["VIF"] >= 5]["variable"]
regression_data = X.drop(high_vif_variables, axis=1)

regression_data.info()

# Regression

## KAN

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X = regression_data
y = data['Price ($)']

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

train_X = torch.tensor(train_X.values, dtype=torch.float32).to(device)
train_y = torch.tensor(train_y.values, dtype=torch.float32).view(-1, 1).to(device)
test_X = torch.tensor(test_X.values, dtype=torch.float32).to(device)
test_y = torch.tensor(test_y.values, dtype=torch.float32).view(-1, 1).to(device)

trainset = TensorDataset(train_X, train_y)

def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    hidden_layers = trial.suggest_int("n_hidden_layers", 2, 5)
    hidden_units = trial.suggest_int("hidden_units", 2, 64)
    batchsize = trial.suggest_int("batchsize", 32, 2048)

    trainloader = DataLoader(trainset, batch_size=batchsize, shuffle=True)

    model = KAN([train_X.shape[1], hidden_units * hidden_layers, 1])
    model.to(torch.float32).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, foreach=False)
    criterion = nn.MSELoss()

    train_loss = 0
    model.train()
    for _, (X, y) in enumerate(trainloader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    return train_loss


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

best_trial = study.best_trial
print("Best Learning Rate:", best_trial.params["learning_rate"])
print("Best Weight Decay:", best_trial.params["weight_decay"])
print("Best Number of Hidden Layers:", best_trial.params["n_hidden_layers"])
print("Best Hidden Units:", best_trial.params["hidden_units"])
print("Best Batch Size:", best_trial.params["batchsize"])

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_param_importances(study)

In [ ]:
best_learning_rate = best_trial.params["learning_rate"]
best_weight_decay = best_trial.params["weight_decay"]
best_hidden_layers = best_trial.params["n_hidden_layers"]
best_hidden_units = best_trial.params["hidden_units"]
best_batchsize = best_trial.params["batchsize"]

trainloader = DataLoader(trainset, batch_size=best_batchsize, shuffle=True)

model = KAN([train_X.shape[1], best_hidden_units * best_hidden_layers, 1])
model.to(torch.float32).to(device)

optimizer = optim.AdamW(model.parameters(), lr=best_learning_rate, weight_decay=best_weight_decay, foreach=False)
criterion = nn.MSELoss()

epochs = 100
train_losses = []

for epoch in range(epochs):
    model.train()
    train_loss = 0
    with tqdm(trainloader, desc=f"Epoch {epoch+1}/{epochs}") as pbar:
        for i, (X, y) in enumerate(pbar):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
    train_losses.append(loss.item())

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(train_losses)
plt.title("Training Loss")

model.eval()

with torch.no_grad():
    y_pred = model(test_X).cpu().numpy()

mse = mean_squared_error(test_y.cpu(), y_pred)
r2 = r2_score(test_y.cpu(), y_pred)
mae = mean_absolute_error(test_y.cpu(), y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R2 Score: {r2}")

## ResNet

In [ ]:
class ResNet(nn.Module):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.num_classes = num_classes
        
        self.resnet = models.resnet18()

        self.resnet.conv1 = nn.Conv2d(self.num_classes, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Linear(512, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.view(-1, self.num_classes, 1, 1)
        x = self.resnet(x)
        return x

In [ ]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    batchsize = trial.suggest_int("batchsize", 32, 2048)

    trainloader = DataLoader(trainset, batch_size=batchsize, shuffle=True)

    model = ResNet(train_X.shape[1])
    model.to(torch.float32).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, foreach=False)
    criterion = nn.MSELoss()

    train_loss = 0
    model.train()
    with tqdm(trainloader, desc="Training") as pbar:
        for _, (X, y) in enumerate(pbar):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

    return train_loss


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

best_trial = study.best_trial
print("Best Learning Rate:", best_trial.params["learning_rate"])
print("Best Weight Decay:", best_trial.params["weight_decay"])
print("Best Batch Size:", best_trial.params["batchsize"])

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_param_importances(study)

In [ ]:
best_learning_rate = best_trial.params["learning_rate"]
best_weight_decay = best_trial.params["weight_decay"]
best_batchsize = best_trial.params["batchsize"]

trainloader = DataLoader(trainset, batch_size=best_batchsize, shuffle=True)

model = ResNet(train_X.shape[1])
model.to(device)
model.float()

optimizer = optim.AdamW(model.parameters(), lr=best_learning_rate, weight_decay=best_weight_decay, foreach=False)
criterion = nn.MSELoss()

epochs = 100
train_losses = []

for epoch in range(epochs):
    model.train()
    train_loss = 0
    with tqdm(trainloader, desc=f"Epoch {epoch+1}/{epochs}") as pbar:
        for i, (X, y) in enumerate(pbar):
            X, y  = X.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
    train_losses.append(loss.item())

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(train_losses)
plt.title("Training Loss")

model.eval()

with torch.no_grad():
    y_pred = model(test_X).cpu().numpy()

mse = mean_squared_error(test_y.cpu(), y_pred)
r2 = r2_score(test_y.cpu(), y_pred)
mae = mean_absolute_error(test_y.cpu(), y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R2 Score: {r2}")

# Linear Sequence Analysis

## ARIMA

In [ ]:
df = pd.read_csv('./data/car_data.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df.head()

In [ ]:
new_df = df.loc[:, ['Date', 'Price ($)']]
new_df.head()

In [ ]:
new_df.fillna(new_df.mean(), inplace = True)

In [ ]:
new_df.set_index('Date', inplace = True)
data_monthly_mean = new_df.resample('M').mean()

In [ ]:
# Moving Average 
# Calculate Simple Moving Average (SMA)
sma_period = 10
new_df['SMA'] = data_monthly_mean['Price ($)'].rolling(window=sma_period).mean().reindex(new_df.index, method='ffill')

# Calculate Exponential Moving Average (EMA)
ema_period = 10
data_monthly_mean['EMA'] = data_monthly_mean['Price ($)'].ewm(span=ema_period, adjust=False).mean()
new_df['EMA'] = data_monthly_mean['EMA'].reindex(new_df.index, method='ffill')

# Calculate Cummulative Moving Average (CMA)
new_df['CMA'] = data_monthly_mean['Price ($)'].expanding(min_periods=1).mean().reindex(new_df.index, method='ffill')

# Calculate Weighted Moving Average (WMA)
wma_period = 10 
weights = pd.Series(range(1, wma_period + 1))
def weighted_moving_average(prices):
    return np.dot(prices, weights) / weights.sum()

new_df['WMA'] = data_monthly_mean['Price ($)'].rolling(window=wma_period).apply(weighted_moving_average, raw=True).reindex(new_df.index, method='ffill')

In [ ]:
plt.figure(figsize = (10, 5))
plt.plot(data_monthly_mean['Price ($)'], label = 'Price')
plt.plot(new_df['SMA'], label = 'SMA')
plt.plot(new_df['EMA'], label = 'EMA')
plt.plot(new_df['CMA'], label = 'CMA')
plt.plot(new_df['WMA'], label = 'WMA')
plt.title('Moving Averages for Monthly Mean Total Price ($)')
plt.xlabel('Year')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

In [ ]:
def adf_test(series):
    adf_statistic, p_value, _, _, critical_values, _= adfuller(series)
    print(f'ADF Statistics: {adf_statistic:.4f}')
    print(f'p-value: {p_value:.4f}')
    print('Critical values:')
    for key, value in critical_values.items():
        print(f'   {key}: {value:.4f}')
    
print("Original Data ADF Test:")
adf_test(data_monthly_mean['Price ($)'])


In [ ]:
plt.figure(figsize = (14, 8))

# ACF plot
plt.subplot(2, 1, 1)
plot_acf(data_monthly_mean['Price ($)'], lags = 10, ax = plt.gca())
plt.title('Autocorrelated Function (ACF)')

# PACF plot
plt.subplot(2, 1, 2)
plot_pacf(data_monthly_mean['Price ($)'], lags = 10, ax = plt.gca())
plt.title('Partial Autocorrelated Function (PACF)')
plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
acf_values = acf(data_monthly_mean['Price ($)'], nlags = 10)
pacf_values = pacf(data_monthly_mean['Price ($)'], nlags = 10)

In [ ]:
# Calculate the 95% confidence interval threshold
n = len(data_monthly_mean['Price ($)'])
threshold = 1.96/np.sqrt(n)

# Count significant values for p and q
significant_p_values = sum(abs(pacf_values[1:]) > threshold)
significant_q_values = sum(abs(acf_values[1:]) > threshold)

print(f"Number significant p values: {significant_p_values}")
print(f"Number significant q values: {significant_q_values}")

In [ ]:
p = 2
d = 0 
q = 1

In [ ]:
model = ARIMA(data_monthly_mean['Price ($)'], order = (p, d, q))
results = model.fit()
print(results.summary())

In [ ]:
residuals = results.resid
plt.figure(figsize = (10, 4))
plt.plot(residuals)
plt.title('Residuals of ARIMA model')
plt.xlabel('Date')
plt.ylabel('Residuals')
plt.show()

In [ ]:
train_size = int(len(data_monthly_mean)*0.8)
train, validation = data_monthly_mean.iloc[:train_size], data_monthly_mean.iloc[train_size:]

In [ ]:
predictions = results.predict(start = validation.index[0], end = validation.index[-1], type = 'levels')

In [ ]:
rmse_score = rmse(validation['Price ($)'], predictions)
print(f'Root Mean Squared Error (RMSE): {rmse_score}')

In [ ]:
plt.figure(figsize = (10, 4))
plt.plot(validation['Price ($)'], label = 'Actual')
plt.plot(predictions, color = 'red', label = 'ARIMA Predicted')
plt.title('ARIMA Model Validation - Actual vs. Predicted')
plt.legend()
plt.show()

In [ ]:
forecast_steps = 12
forecast = results.get_forecast(steps = forecast_steps)
predicted_values = forecast.predicted_mean
print(predicted_values)

In [ ]:
plt.figure(figsize = (15, 6))
plt.plot(data_monthly_mean['Price ($)'], label = 'Original')
plt.plot(predicted_values.index, predicted_values.values, color = 'red', label = 'Predicted')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.title('ARIMA Predictions from Price Data')
plt.legend()
plt.show()

In [ ]:
data = data.values.astype(float).reshape(-1)

## xLSTM Model

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from xlstm import xLSTM

# Define the dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, index):
        if index + self.seq_len >= len(self.data):
            raise IndexError("Index out of range for dataset")
        x = self.data[index:index + self.seq_len]
        y = self.data[index + self.seq_len]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# # Load the data
# def load_data(file_path):
#     df = pd.read_csv(file_path, usecols=[1])
#     data = df.values.astype(float).reshape(-1)
#     return data

# Define the evaluation function for validation
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            output, _ = model(x_batch.unsqueeze(-1))
            loss = criterion(output[-1], y_batch)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Define the evaluation function for full sequence
def evaluate_model_full_sequence(model, data, device, seq_len):
    model.eval()
    predictions = []
    data = torch.tensor(data, dtype=torch.float32).unsqueeze(-1).to(device)
    with torch.no_grad():
        for i in range(seq_len, len(data)):
            input_seq = data[i-seq_len:i].unsqueeze(1)  # Shape: (seq_len, 1, 1)
            output, _ = model(input_seq)
            prediction = output[-1].item()
            predictions.append(prediction)
    return predictions

# Define the training function
def train_model(model, train_dataloader, val_dataloader, criterion, optimizer, device, num_epochs=20):
    train_losses = []
    val_losses = []
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output, _ = model(x_batch.unsqueeze(-1))
            loss = criterion(output[-1], y_batch)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        train_losses.append(total_train_loss / len(train_dataloader))

        # Evaluate on validation set
        val_loss = evaluate_model(model, val_dataloader, criterion, device)
        val_losses.append(val_loss)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_losses[-1]}, Val Loss: {val_losses[-1]}")
    return train_losses, val_losses

# Parameters
# file_path = 'AirPassengers.csv'  # Change this to your dataset path
seq_len = 16
batch_size = 512
hidden_size = 128
num_heads = 1
layers = ['m', 's', 'm']
learning_rate = 0.01
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load data
# data = load_data(file_path)
dataset = TimeSeriesDataset(data, seq_len)

# Split data into training, validation, and test sets
train_size = int(len(dataset) * 0.6)
val_size = int(len(dataset) * 0.2)
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define model
model = xLSTM(input_size=1, hidden_size=hidden_size, num_heads=num_heads, layers=layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
# Train model
train_losses, val_losses = train_model(model, train_dataloader, val_dataloader, criterion, optimizer, device, num_epochs)

# Evaluate model on the test set
test_data = data[-(test_size + seq_len):]
predictions = evaluate_model_full_sequence(model, test_data, device, seq_len)

# Plot the results
plt.figure(figsize=(14, 6))
plt.subplot(2, 1, 1)
plt.plot(data, label='Actual')
plt.plot(range(len(data) - test_size, len(data)), predictions, label='Predicted')
plt.xlabel('Time')
plt.ylabel('Number of Passengers')
plt.legend()

# Plot training and validation loss
plt.subplot(2, 1, 2)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()
